In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_40.csv')
pred2 = dd.read_csv('test_pred_41.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 33.8 s, sys: 3.4 s, total: 37.2 s
Wall time: 8.02 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 262 ms, sys: 307 ms, total: 569 ms
Wall time: 565 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 282 µs, sys: 38 µs, total: 320 µs
Wall time: 326 µs


In [7]:
coef_lgb = 0.85
coef_mlp = 0.15

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 332 ms, sys: 583 ms, total: 916 ms
Wall time: 912 ms


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [10]:
test_ensemble['object_id'] = pred1['object_id']
print(test_ensemble.shape)

(3492890, 16)


In [11]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [12]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000183,0.373841,0.000369,0.198529,0.105222,0.000421,0.130000,0.000257,0.001557,0.016144,0.002258,0.170368,0.000270,0.000578,0.245496
3492886,130787971,0.000198,0.012471,0.000168,0.050983,0.163894,0.000238,0.021066,0.390383,0.000381,0.036875,0.000775,0.315767,0.000255,0.006543,0.234137
3492887,130787974,0.000750,0.022835,0.000141,0.863000,0.032679,0.000339,0.066587,0.000045,0.000150,0.001307,0.000383,0.009529,0.000202,0.002053,0.095064
3492888,130788053,0.000061,0.717338,0.000078,0.153662,0.003308,0.000114,0.010514,0.084340,0.000446,0.000352,0.000226,0.028828,0.000104,0.000629,0.165102
3492889,130788054,0.000209,0.088182,0.000306,0.422389,0.188065,0.000506,0.059341,0.000103,0.000537,0.022792,0.001190,0.214889,0.000674,0.000818,0.236735


In [13]:
%%time
test_ensemble.to_csv('test_pred_44.csv',index=False)

CPU times: user 1min 32s, sys: 1.29 s, total: 1min 34s
Wall time: 1min 34s


In [14]:
test_ensemble['object_id'].equals(pred1['object_id'])

True

In [15]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_44.csv -m "Message"

100%|██████████████████████████████████████| 1.06G/1.06G [04:04<00:00, 4.67MB/s]
Successfully submitted to PLAsTiCC Astronomical Classification